# Data Preprocessing Homework

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("momentum_strategy_1.csv")
df.head()

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,FANG,$102.81,167.05%,98.42%,110.39%,99.80%,23.90%,95.84%,15.58%,97.23%,97.82%,786
1,NVDA,$836.32,117.59%,92.67%,57.08%,96.24%,47.60%,100.00%,25.29%,99.60%,97.13%,96
2,DXC,$42.36,156.16%,97.43%,58.69%,96.83%,31.47%,99.01%,6.28%,83.56%,94.21%,1909
3,OXY,$32.72,92.75%,85.54%,91.00%,99.21%,20.26%,92.28%,15.90%,97.43%,93.61%,2472
4,IT,$258.87,107.52%,91.09%,57.23%,96.44%,32.85%,99.60%,6.90%,84.75%,92.97%,312


In [3]:
df.isna().any()

Ticker                           False
Price                            False
One Year Price Return            False
One Year Return Percentile       False
Six Month Price Return            True
Six Month Return Percentile      False
Three Month Price Return          True
Three Month Return Percentile    False
One Month Price Return           False
One Month Return Percentile      False
Momentum Score                   False
Number of Shares to Buy          False
dtype: bool

Since we have NaNs in returns *only*, we shall drop all rows containing them because we can't accurately estimate them.

In [4]:
df = df.dropna().reset_index(drop=True)

We lost 1 ticker. Now we shall process the percentages.

In [5]:
column_names = list(df)
column_names.pop(0)
column_names.pop(0)
column_names.pop(-1)
column_names

['One Year Price Return',
 'One Year Return Percentile',
 'Six Month Price Return',
 'Six Month Return Percentile',
 'Three Month Price Return',
 'Three Month Return Percentile',
 'One Month Price Return',
 'One Month Return Percentile',
 'Momentum Score']

In [6]:
for name in column_names:
    df[name] = df[name].str.rstrip('%').astype('float') / 100.0

USD to `float`

In [7]:
for price in enumerate(df.Price):
    if price[1].find("$"):
        print(price)

(17, 'USD2678.61')


In [8]:
df.Price = df.Price.map(lambda x: float(x.replace("USD", "$").replace("$", "")))

Finally, the data is ready for some descriptive analysis.

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,51.0,285.435686,515.626100,13.9700,63.27500,121.6000,276.69000,2678.6100
One Year Price Return,51.0,1.041796,0.565323,0.3213,0.70365,0.9275,1.20490,3.9886
One Year Return Percentile,51.0,7.347116,46.560530,0.4911,0.72770,0.8634,0.94255,333.3347
Six Month Price Return,51.0,0.277547,1.540648,-10.3919,0.31895,0.4137,0.58840,1.1628
Six Month Return Percentile,51.0,0.872841,0.114821,0.5505,0.80100,0.9050,0.96930,1.0000
Three Month Price Return,51.0,0.205684,0.073345,0.0683,0.16660,0.1904,0.24400,0.4760
Three Month Return Percentile,51.0,0.888602,0.101527,0.5624,0.86235,0.9089,0.96240,1.0000
One Month Price Return,51.0,0.071139,0.067901,-0.0103,0.02165,0.0610,0.09470,0.3066
One Month Return Percentile,51.0,0.789204,0.159274,0.4574,0.67525,0.8317,0.91390,1.0000
Momentum Score,51.0,0.842088,0.054827,0.7762,0.79625,0.8317,0.87180,0.9782


From the descriptive statistics table and the meaning of the given data we can clearly notice a few more problems with the dataset:

1. `One Year Return Percentile` has a maximum value of 333.3347, which is not possible due to the percentile definition;
2. `Six Month Price Return` has a minimum value of -10.3919. The minimal possible value is -1.0 (when the stock price goes to zero);
3. Bath & Body Works Inc (`LB` ticker) has an annual return of 398.86%, which is strange, but given the pandemic, the strangeness disappears.

We can extrapolate the problematic values for the percentiles and returns, but it not worth the effort (due to the size of the given dataset). Therefore, we shall drop them.

In [10]:
df = df[df["One Year Return Percentile"] <= 1.0]
df = df[df["Six Month Price Return"] >= -1.0]

The final dataset looks the following way:

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,49.0,284.944694,526.130657,13.9700,61.7300,102.8100,258.8700,2678.6100
One Year Price Return,49.0,1.068535,0.560461,0.4792,0.7146,0.9505,1.2170,3.9886
One Year Return Percentile,49.0,0.834227,0.126672,0.5267,0.7287,0.8634,0.9386,1.0000
Six Month Price Return,49.0,0.492065,0.231070,0.2080,0.3245,0.4137,0.5899,1.1628
Six Month Return Percentile,49.0,0.871653,0.116923,0.5505,0.7980,0.9050,0.9703,1.0000
Three Month Price Return,49.0,0.203610,0.074103,0.0683,0.1644,0.1844,0.2390,0.4760
Three Month Return Percentile,49.0,0.885106,0.102073,0.5624,0.8574,0.9030,0.9584,1.0000
One Month Price Return,49.0,0.064302,0.058253,-0.0103,0.0166,0.0600,0.0871,0.2529
One Month Return Percentile,49.0,0.780884,0.156911,0.4574,0.6475,0.8297,0.8970,0.9960
Momentum Score,49.0,0.842967,0.055731,0.7762,0.7955,0.8361,0.8743,0.9782


In [12]:
df

,Ticker,Price,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,Momentum Score,Number of Shares to Buy
0,FANG,102.81,1.6705,0.9842,1.1039,0.9980,0.2390,0.9584,0.1558,0.9723,0.9782,786
1,NVDA,836.32,1.1759,0.9267,0.5708,0.9624,0.4760,1.0000,0.2529,0.9960,0.9713,96
2,DXC,42.36,1.5616,0.9743,0.5869,0.9683,0.3147,0.9901,0.0628,0.8356,0.9421,1909
3,OXY,32.72,0.9275,0.8554,0.9100,0.9921,0.2026,0.9228,0.1590,0.9743,0.9361,2472
4,IT,258.87,1.0752,0.9109,0.5723,0.9644,0.3285,0.9960,0.0690,0.8475,0.9297,312
5,FTNT,251.21,0.7522,0.7683,0.6590,0.9802,0.3115,0.9861,0.1332,0.9564,0.9228,322
6,LB,76.99,3.9886,1.0000,1.0050,0.9960,0.2022,0.9208,0.0418,0.7703,0.9218,1050
7,WAT,359.10,0.9571,0.8673,0.4362,0.9287,0.2416,0.9604,0.1055,0.9228,0.9198,225
8,MRO,13.97,1.4158,0.9683,1.1628,1.0000,0.2086,0.9287,0.0304,0.7248,0.9054,5790
9,TGT,256.84,1.0793,0.9129,0.3878,0.8733,0.2271,0.9465,0.0728,0.8693,0.9005,314
